In [ ]:
import pandas as pd
import json
from tqdm import tqdm

In [ ]:
sol = pd.read_csv('solutions_week2.csv', sep=';').iloc[:,[0,-1]]
solutions = [json.loads(i) for i in sol['solution']]
sol['solution'] = solutions
sol['week'] = [3 if '3' in i[:i.index('/')] else 2 for i in sol['instance']]
# instance -> String: <CLASS>/<NAME>
# solution -> Array<Integer>: All vertices that have not been root clean
# week -> int: Week indicator either 2 or 3
sol = sol.sort_values(by='instance')

vert = pd.read_csv('vertices_week2_with_deleted.csv')

In [ ]:
vert